# Local Variational Inference for a Softmax Model

The local variational method can be extended to multidimensional models by use of the softmax function (see e.g. Ahmed, 2013). In this demo we consider the following model:

\begin{align*}
    x_t &\sim \mathcal{N}(A x_{t-1}, 0.01 I)\\
    y_t &\sim \mathcal{C}at(\sigma(x_t))\,,
\end{align*}

with $\sigma$ a softmax. In this demo, we extend upon this by using the softmax to implement a "greater than" constraint as used in the example from Infer.NET, 2020, see references. The example consists of a number of match results in head-to-head encounters between 5 players being used to estimate their (relative) skills, including an estimate of the uncertainty on each skill. This notebook was developed by Keith Myerscough of Sioux LIME, lending heavily from other notebooks in this demo folder. 

In [ ]:
using Random
using ForneyLab
using PyPlot
using LinearAlgebra

# Generate data set
Random.seed!(21)
σ(x) = exp.(x)/sum(exp.(x)) # Softmax function

n_players = 5
winners = [0, 0, 0, 1, 3, 4] .+ 1  # .+ 1 for Julia-indexing
losers = [1, 3, 4, 2, 1, 2] .+ 1
n_matches = length(winners)
;

# Model specification

The model specification includes local variational parameters `xi` and `a`, which are used to define an upperbound on the softmax (Bouchard, 2007).

In [ ]:
g = FactorGraph()

perf_var = 1
perf_prec = 1/perf_var^2

m_s_prior = 6. * ones(n_players)
v_s_prior = 9. * eye(n_players)

width = 2

@RV s ~ GaussianMeanVariance(m_s_prior, v_s_prior)
p = Vector{Variable}(undef, n_matches)
xi = Vector{Variable}(undef, n_matches)
a = Vector{Variable}(undef, n_matches)
y = Vector{Variable}(undef, n_matches)

for i_m = 1:n_matches
    println("$(winners[i_m]) beats $(losers[i_m])")
    A = zeros(2, n_players)
    A[1, winners[i_m]] = 1
    A[2, losers[i_m]] = 1
#     println(A)
    @RV p[i_m] ~ GaussianMeanPrecision(A * s, perf_prec*eye(2))
    @RV xi[i_m]
    @RV a[i_m]
    @RV y[i_m] ~ Softmax(p[i_m], xi[i_m], a[i_m])

    # Data placeholder
    placeholder(y[i_m], :y, index=i_m, dims=(2,))
end
;

In [ ]:
ForneyLab.draw(g)

# Algorithm generation

Since we are interested in optimizing the local variational parameters `xi`, `a` together with the hidden state sequence `x`, we construct an algorithm that also updates `xi` and `a`. We can also generate an algorithm for evaluating the free energy. However, because we upper-bound the softmax, the free energy is no longer guaranteed to be a upper bound on surprise. This is in contrast to local variational estimation for the logistic function, which is lower bounded (see the corresponding demo).

In [ ]:
algo = variationalAlgorithm(s, p, xi, a, ids=[:S, :P, :Xi, :A], free_energy=true)
source_code = algorithmSourceCode(algo, free_energy=true);

In [ ]:
# println(source_code) # Uncomment to inspect algorithm code

# Execution

For execution we initialize the local variational parameters and iterate the automatically derived algorithm.

In [ ]:
eval(Meta.parse(source_code));

In [ ]:
# Pre-initialize marginals
marginals = Dict()

marginals[:s] = ProbabilityDistribution(Multivariate, GaussianMeanVariance, 
    m=m_s_prior, v=v_s_prior)
for t=1:n_matches
    marginals[:p_*t] = ProbabilityDistribution(Multivariate, GaussianMeanVariance, 
        m=m_s_prior[1] * ones(2), v=v_s_prior[1, 1] * eye(2))
    marginals[:xi_*t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=1. * ones(2), w=1. * eye(2))
    marginals[:a_*t] = ProbabilityDistribution(Univariate, GaussianMeanPrecision)
end
data = Dict(:y  => [[1, 0] for i_m=1:n_matches])

n_its = 100
F = Vector{Float64}(undef, n_its)

println("$(mean(marginals[:s])[1]) ± $(var(marginals[:s])[1])")
for i = 1:n_its
    stepA!(data, marginals)
    stepXi!(data, marginals) # Update local variational parameters
    stepS!(data, marginals) # Update hidden state
    stepP!(data, marginals) # Update hidden state
end

# Results

Results show that the algorithm accurately estimates the hidden state.

In [ ]:
# Extract posterior state statistics
m_s = [mean(marginals[:s])[t] for t = 1:n_players]
v_s = [cov(marginals[:s])[t, t] for t = 1:n_players]
m_p_1 = [mean(marginals[:p_*t])[1] for t = 1:n_matches]
v_p_1 = [cov(marginals[:p_*t])[1, 1] for t = 1:n_matches]
m_p_2 = [mean(marginals[:p_*t])[2] for t = 1:n_matches]
v_p_2 = [cov(marginals[:p_*t])[2, 2] for t = 1:n_matches]
;

In [ ]:
order = sortperm(m_s - 3*v_s)
for i_p=n_players:-1:1
    j_p = order[i_p]
    println("player $(j_p-1) with rating $(m_s[j_p]) ± $(3*sqrt.(v_s[j_p]))")
end

## Comparing to reference

The output is compared to that of Infer.NET, 2020.

In [ ]:
ref_mean = [9.517, 4.955, 2.639, 6.834, 6.054]
ref_dev = [3.926, 3.503, 4.288, 3.892, 4.731]

In [ ]:
plt.scatter(ref_mean, m_s)
plt.plot([2, 10], [2, 10])
m_s ./ ref_mean

In [ ]:
plt.scatter(ref_dev, v_s)
v_s, ref_dev

While the esimated skills are very close to the reference of Infer.NET, the variances are not. This might be due to the current results being based on Variance Message Passing, while Infer.NET uses Expectation Propagation. The key difference is the order of arguments in the underlying Kullback-Leibler divergence. Using Expectation Propagation in ForneyLab has not proven successful yet; this is work-in-progress.

### References

Bouchard, 2007 "Efficient Bounds for the Softmax Function"

Ahmed, 2013, "Bayesian Multicategorical Soft Data Fusion for Human-Robot Collaboration"

Infer.NET, 2020, "https://docs.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/matchup-app-infer-net"